## iPyLeaflet
[ipyleaflet](https://github.com/ellisonbg/ipyleaflet) is a bridge between jupyter notebooks and the [leaflet](http://leafletjs.com/)  javascript library for drawing maps.

ipyleaflet comes with a few examples notebooks (this notebook was derived from one) but very little documentation,
for more documentation read the [Leaflet IPA](http://leafletjs.com/reference.html)

For installation directions, see the README on [ipyleaflet](https://github.com/ellisonbg/ipyleaflet)

[GeoJson](https://tools.ietf.org/html/rfc7946#section-3.1.7) is a good way to add data on top of an ipyleaflet map.

For other backgrounds than open streets, you can alter the ipyleaflet.py file to point to one of [these map providers](https://leaflet-extras.github.io/leaflet-providers/preview/)

In [53]:
from ipyleaflet import (
    Map,
    Marker,
    TileLayer, ImageOverlay,
    Polyline, Polygon, Rectangle, Circle, CircleMarker,
    GeoJSON,
    DrawControl
)

## Visualizing the distribution of the observations

## Load the required libraries

In [54]:
import pandas as pd
import numpy as np
import sklearn as sk
import urllib
import math
import pylab as plt

import sys
sys.path.append('./lib')

from leaflet import *

In [59]:
data_dir = "../../Data/Weather"
from pickle import load
with open(data_dir+'/PRCP_residuals_PCA.pickle','rb') as file:
    Res_eig=load(file)
print Res_eig.keys()

#stations.pkl
with open(data_dir+'/stations.pkl','rb') as file:
    Stations=load(file)
Stations.head()


['eigen-vecs', 'P_norm', 'stations']


,latitude,longitude,elevation,state,name,GSNFLAG,HCNFLAG,WMOID
ACW00011604,17.1167,-61.7833,10.1,NaN,ST JOHNS COOLIDGE FLD,NaN,NaN,NaN
ACW00011647,17.1333,-61.7833,19.2,NaN,ST JOHNS,NaN,NaN,NaN
AE000041196,25.3330,55.5170,34.0,NaN,SHARJAH INTER. AIRP,GSN,NaN,41196.0
AF000040930,35.3170,69.0170,3366.0,NaN,NORTH-SALANG,GSN,NaN,40930.0
AG000060390,36.7167,3.2500,24.0,NaN,ALGER-DAR EL BEIDA,GSN,NaN,60390.0


In [56]:
P_norm = Res_eig['P_norm']

In [60]:
import pandas as pd
Eig=pd.DataFrame(Res_eig['eigen-vecs'].transpose())
P_norm = pd.DataFrame(P_norm.transpose())
# print Eig.head()
# print P_norm.head(122)

In [61]:
Eig['station']=Res_eig['stations']
Eig=Eig.set_index('station')
print Res_eig['stations'][:5]
Eig.head()

[u'CA004010080', u'CA004014913', u'CA005011080', u'CA004012485', u'CA005010760']


,0,1,2,3,4,5,6,7,8,9
station,,,,,,,,,,
CA004010080,0.011866,0.022248,-0.060888,0.054217,-0.041003,-0.037614,0.093085,0.047588,0.101305,0.005873
CA004014913,-0.129668,-0.013274,0.095793,0.148383,-0.174928,0.063966,0.068865,0.067242,-0.087085,0.077272
CA005011080,-0.089174,0.053184,-0.039168,0.104050,-0.134874,0.115964,-0.037200,-0.048276,-0.027019,0.123875
CA004012485,0.017531,-0.033007,-0.029974,0.011458,0.022773,-0.042381,0.109229,0.103034,0.053905,0.047253
CA005010760,-0.087771,0.061720,-0.042258,0.129494,-0.120663,0.129413,-0.032053,-0.098141,-0.071421,0.055561


In [62]:
table=Eig.join(Stations,how='left')

In [63]:
table=table[['name','latitude','longitude','elevation',0,1,2,3]]
table.head(4)

,name,latitude,longitude,elevation,0,1,2,3
station,,,,,,,,
CA004010080,ALAMEDA CDA EPF,49.25,-102.28,580.0,0.011866,0.022248,-0.060888,0.054217
CA004014913,MANOR,49.62,-102.10,633.0,-0.129668,-0.013274,0.095793,0.148383
CA005011080,GOODLANDS CDA EPF,49.13,-100.60,500.0,-0.089174,0.053184,-0.039168,0.104050
CA004012485,FERTILE,49.33,-101.45,511.0,0.017531,-0.033007,-0.029974,0.011458


In [87]:
table_spatial = table.sort([0],ascending=[1]).head(20)
table_spatial.head()

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


,name,latitude,longitude,elevation,0,1,2,3
station,,,,,,,,
USC00324545,KARLSRUHE 6N,48.1378,-100.5397,451.1,-0.174918,-0.009941,0.140428,0.013831
USC00324879,LAKE METIGOSHE SP,48.9797,-100.3344,652.9,-0.171453,0.007180,0.148411,-0.020403
CA005010548,CARBERRY,49.8700,-99.3500,385.0,-0.170263,-0.041850,0.151071,0.040593
USC00327201,PORTAL,49.0000,-102.5500,595.0,-0.168217,-0.028277,0.164639,0.082865
USC00328792,TOWNER 2 NE,48.3706,-100.3908,451.1,-0.163728,0.067626,-0.039866,-0.004942


In [81]:
import pylab as plt
prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']

## Map

In [89]:
min_lat,max_lat,min_long,max_long = box = (48.0475, 49.98, -102.55, -98.9167)
center = [(min_lat+max_lat)/2, (min_long+max_long)/2]
zoom = 9
m = Map(default_tiles=TileLayer(opacity=0.6), center=center, zoom=zoom)
r = Rectangle(bounds=[[min_lat,min_long],[max_lat,max_long]], weight=5, fill_opacity=0.0)
m += r

lat_margin=(max_lat-min_lat)/4
long_margin=(max_long-min_long)/4
for index,row in table_spatial.iterrows():
    _lat=row['latitude']
    _long=row['longitude']
    for col in range(4):
        _coef=row[col]
        if np.isnan(_coef):
            continue
        r=abs(_coef)/5
        #print r
        #color=colors[col]
        signs=[[+1,+1],[+1,-1],[-1,-1],[-1,+1]]
        lat_sign,long_sign=signs[col]
        triangle=[(_lat,_long),(_lat+lat_sign*r,_long),(_lat,_long+long_sign*r),(_lat,_long)]
        if _coef<0:
            color = '#F00'
            poly= Polygon(locations=triangle, weight=0,
                    color=color, opacity=0, fill_opacity=0.7,
                    fill_color=color)
        else:
            color = '#F00'
            poly= Polygon(locations=triangle, weight=2,
                    color=color, opacity=0.8, fill_opacity=0,
                    fill_color=color)
        m +=poly   
m    

### excercises:
* Add a legend that relates the colors to values.
* Leaflet supports a variety of maps. See if you can get a topographical map as the background.

In [23]:
%matplotlib inline
pdf.plot.scatter(x='elevation',y='avg(coeff_1)');

NameError: name 'pdf' is not defined